In [61]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from matplotlib import dates
import plotly.graph_objs as go
import plotly.offline as pyoff

#import Keras
from tensorflow.keras.optimizers import Adam 
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from keras.layers import LSTM
from sklearn.model_selection import KFold, cross_val_score, train_test_split
import statsmodels.formula.api as smf

In [62]:
df = pd.read_excel('itService_day.xlsx')
df['RQST_DT'] = df['RQST_DT'].astype(str)
df['RQST_DT'] = pd.to_datetime(df['RQST_DT'])
df.rename(columns = {'RQST_DT' : 'YYMM'}, inplace = True)
df


,YYMM,CNT
0,2014-01-02,5
1,2014-01-03,4
2,2014-01-06,8
3,2014-01-07,6
4,2014-01-08,2
...,...,...
1731,2021-12-24,2
1732,2021-12-28,6
1733,2021-12-29,3
1734,2021-12-30,5


In [63]:
#create a new dataframe to model the difference
df_diff = df.copy()
#add previous sales to the next row
df_diff['prev_CNT'] = df_diff['CNT'].shift(1)
#drop the null values and calculate the difference
df_diff = df_diff.dropna()
df_diff['diff'] = (df_diff['CNT'] - df_diff['prev_CNT'])
df_diff.head(10)


,YYMM,CNT,prev_CNT,diff
1,2014-01-03,4,5.0,-1.0
2,2014-01-06,8,4.0,4.0
3,2014-01-07,6,8.0,-2.0
4,2014-01-08,2,6.0,-4.0
5,2014-01-09,2,2.0,0.0
6,2014-01-10,9,2.0,7.0
7,2014-01-13,7,9.0,-2.0
8,2014-01-14,3,7.0,-4.0
9,2014-01-15,4,3.0,1.0
10,2014-01-16,8,4.0,4.0


In [64]:
df_diff = df_diff[df['YYMM'].between('2021-01-01', '2021-12-31')]
df_diff

C:\Users\admin\AppData\Local\Temp/ipykernel_25348/1285525311.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,YYMM,CNT,prev_CNT,diff
1481,2021-01-04,4,2.0,2.0
1482,2021-01-05,11,4.0,7.0
1483,2021-01-06,6,11.0,-5.0
1484,2021-01-07,6,6.0,0.0
1485,2021-01-08,1,6.0,-5.0
...,...,...,...,...
1731,2021-12-24,2,10.0,-8.0
1732,2021-12-28,6,2.0,4.0
1733,2021-12-29,3,6.0,-3.0
1734,2021-12-30,5,3.0,2.0


In [65]:
#plot sales diff
plot_data = [
    go.Scatter(
        x=df_diff['YYMM'],
        y=df_diff['CNT'],
    )
]
plot_layout = go.Layout(
        title='ITService Count'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [66]:
#plot sales diff
plot_data = [
    go.Scatter(
        x=df_diff['YYMM'],
        y=df_diff['diff'],
    )
]
plot_layout = go.Layout(
        title='ITService Diff'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [67]:
#create dataframe for transformation from time series to supervised

df_supervised = df_diff.drop(['prev_CNT'],axis=1)
#adding lags
for inc in range(1,51):
    field_name = 'lag_' + str(inc)
    df_supervised[field_name] = df_supervised['diff'].shift(inc)
#drop null values
df_supervised = df_supervised.dropna().reset_index(drop=True)

In [68]:
df_supervised.head(10)

,YYMM,CNT,diff,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,...,lag_41,lag_42,lag_43,lag_44,lag_45,lag_46,lag_47,lag_48,lag_49,lag_50
0,2021-03-15,15,-2.0,7.0,-5.0,-11.0,7.0,5.0,-21.0,16.0,...,4.0,-2.0,-2.0,0.0,4.0,-5.0,0.0,-5.0,7.0,2.0
1,2021-03-16,29,14.0,-2.0,7.0,-5.0,-11.0,7.0,5.0,-21.0,...,-2.0,4.0,-2.0,-2.0,0.0,4.0,-5.0,0.0,-5.0,7.0
2,2021-03-17,19,-10.0,14.0,-2.0,7.0,-5.0,-11.0,7.0,5.0,...,-1.0,-2.0,4.0,-2.0,-2.0,0.0,4.0,-5.0,0.0,-5.0
3,2021-03-18,17,-2.0,-10.0,14.0,-2.0,7.0,-5.0,-11.0,7.0,...,0.0,-1.0,-2.0,4.0,-2.0,-2.0,0.0,4.0,-5.0,0.0
4,2021-03-19,12,-5.0,-2.0,-10.0,14.0,-2.0,7.0,-5.0,-11.0,...,1.0,0.0,-1.0,-2.0,4.0,-2.0,-2.0,0.0,4.0,-5.0
5,2021-03-22,8,-4.0,-5.0,-2.0,-10.0,14.0,-2.0,7.0,-5.0,...,0.0,1.0,0.0,-1.0,-2.0,4.0,-2.0,-2.0,0.0,4.0
6,2021-03-23,10,2.0,-4.0,-5.0,-2.0,-10.0,14.0,-2.0,7.0,...,-2.0,0.0,1.0,0.0,-1.0,-2.0,4.0,-2.0,-2.0,0.0
7,2021-03-24,16,6.0,2.0,-4.0,-5.0,-2.0,-10.0,14.0,-2.0,...,0.0,-2.0,0.0,1.0,0.0,-1.0,-2.0,4.0,-2.0,-2.0
8,2021-03-25,20,4.0,6.0,2.0,-4.0,-5.0,-2.0,-10.0,14.0,...,5.0,0.0,-2.0,0.0,1.0,0.0,-1.0,-2.0,4.0,-2.0
9,2021-03-26,6,-14.0,4.0,6.0,2.0,-4.0,-5.0,-2.0,-10.0,...,-4.0,5.0,0.0,-2.0,0.0,1.0,0.0,-1.0,-2.0,4.0


In [70]:
#import MinMaxScaler and create a new dataframe for LSTM model
from sklearn.preprocessing import MinMaxScaler
df_model = df_supervised.drop(['CNT','YYMM'],axis=1)
#split train and test set
train_set, test_set = df_model[0:-6].values, df_model[-6:].values

In [71]:
df_model[0:-6].values

array([[ -2.,   7.,  -5., ...,  -5.,   7.,   2.],
       [ 14.,  -2.,   7., ...,   0.,  -5.,   7.],
       [-10.,  14.,  -2., ...,  -5.,   0.,  -5.],
       ...,
       [  3.,  -3.,  -5., ...,   1.,  -5.,   5.],
       [  4.,   3.,  -3., ...,  -2.,   1.,  -5.],
       [ -2.,   4.,   3., ...,   4.,  -2.,   1.]])

In [72]:
#apply Min Max Scaler
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler = scaler.fit(train_set)
# reshape training set
train_set = train_set.reshape(train_set.shape[0], train_set.shape[1])
train_set_scaled = scaler.transform(train_set)
# reshape test set
test_set = test_set.reshape(test_set.shape[0], test_set.shape[1])
test_set_scaled = scaler.transform(test_set)

In [73]:
X_train, y_train = train_set_scaled[:, 1:], train_set_scaled[:, 0:1]
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test, y_test = test_set_scaled[:, 1:], test_set_scaled[:, 0:1]
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

In [74]:
X_train.shape

(199, 1, 50)

In [75]:
model = Sequential()
model.add(LSTM(32, batch_input_shape=(1, X_train.shape[1], X_train.shape[2]), stateful=True))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, epochs=500, batch_size=1, verbose=1, shuffle=False)

Epoch 1/500
199/199 [==============================] - 1s 829us/step - loss: 0.0860
Epoch 2/500
199/199 [==============================] - 0s 809us/step - loss: 0.0666
Epoch 3/500
199/199 [==============================] - 0s 784us/step - loss: 0.0521
Epoch 4/500
199/199 [==============================] - 0s 789us/step - loss: 0.0430
Epoch 5/500
199/199 [==============================] - 0s 773us/step - loss: 0.0367
Epoch 6/500
199/199 [==============================] - 0s 778us/step - loss: 0.0318
Epoch 7/500
199/199 [==============================] - 0s 784us/step - loss: 0.0279
Epoch 8/500
199/199 [==============================] - 0s 784us/step - loss: 0.0245
Epoch 9/500
199/199 [==============================] - 0s 799us/step - loss: 0.0216
Epoch 10/500
199/199 [==============================] - 0s 773us/step - loss: 0.0189
Epoch 11/500
199/199 [==============================] - 0s 789us/step - loss: 0.0170
Epoch 12/500
199/199 [==============================] - 0s 789us/step - lo

In [76]:
model.evaluate(X_test, y_test)

1/1 [==============================] - 0s 254ms/step - loss: 0.0847


0.08469211310148239

In [77]:
y_pred = model.predict(X_test,batch_size=1)
X_test

array([[[-0.09090909,  0.27272727,  0.21212121, -0.15151515,
         -0.27272727,  0.18918919,  0.24324324,  0.02702703,
          0.1627907 , -0.34883721,  0.20930233, -0.11627907,
          0.06976744, -0.02325581, -0.20930233,  0.02325581,
          0.30232558, -0.30232558,  0.1627907 ,  0.02325581,
         -0.30232558,  0.11627907, -0.11627907,  0.11627907,
         -0.02325581, -0.02325581, -0.06976744,  0.02325581,
         -0.06976744,  0.1627907 , -0.34883721, -0.06976744,
          0.02325581,  0.1627907 , -0.20930233,  0.1627907 ,
         -0.30232558, -0.02325581,  0.25581395, -0.20930233,
          0.06976744,  0.06976744, -0.11627907,  0.06976744,
         -0.02325581, -0.11627907, -0.02325581, -0.1627907 ,
          0.1627907 , -0.11627907]],

       [[ 0.27272727, -0.09090909,  0.27272727,  0.21212121,
         -0.15151515, -0.13513514,  0.18918919,  0.24324324,
         -0.11627907,  0.1627907 , -0.34883721,  0.20930233,
         -0.11627907,  0.06976744, -0.02325581,

In [78]:
#reshape y_pred
y_pred = y_pred.reshape(y_pred.shape[0], 1, y_pred.shape[1])
#rebuild test set for inverse transform
pred_test_set = []
for index in range(0,len(y_pred)):
    print(np.concatenate([y_pred[index],X_test[index]],axis=1))
    pred_test_set.append(np.concatenate([y_pred[index],X_test[index]],axis=1))
#reshape pred_test_set
pred_test_set = np.array(pred_test_set)
pred_test_set = pred_test_set.reshape(pred_test_set.shape[0], pred_test_set.shape[2])
#inverse transform
pred_test_set_inverted = scaler.inverse_transform(pred_test_set)
pred_test_set


[[ 0.06095932 -0.09090909  0.27272727  0.21212121 -0.15151515 -0.27272727
   0.18918919  0.24324324  0.02702703  0.1627907  -0.34883721  0.20930233
  -0.11627907  0.06976744 -0.02325581 -0.20930233  0.02325581  0.30232558
  -0.30232558  0.1627907   0.02325581 -0.30232558  0.11627907 -0.11627907
   0.11627907 -0.02325581 -0.02325581 -0.06976744  0.02325581 -0.06976744
   0.1627907  -0.34883721 -0.06976744  0.02325581  0.1627907  -0.20930233
   0.1627907  -0.30232558 -0.02325581  0.25581395 -0.20930233  0.06976744
   0.06976744 -0.11627907  0.06976744 -0.02325581 -0.11627907 -0.02325581
  -0.1627907   0.1627907  -0.11627907]]
[[-0.17410956  0.27272727 -0.09090909  0.27272727  0.21212121 -0.15151515
  -0.13513514  0.18918919  0.24324324 -0.11627907  0.1627907  -0.34883721
   0.20930233 -0.11627907  0.06976744 -0.02325581 -0.20930233  0.02325581
   0.30232558 -0.30232558  0.1627907   0.02325581 -0.30232558  0.11627907
  -0.11627907  0.11627907 -0.02325581 -0.02325581 -0.06976744  0.0232558

array([[ 0.06095932, -0.09090909,  0.27272727,  0.21212121, -0.15151515,
        -0.27272727,  0.18918919,  0.24324324,  0.02702703,  0.1627907 ,
        -0.34883721,  0.20930233, -0.11627907,  0.06976744, -0.02325581,
        -0.20930233,  0.02325581,  0.30232558, -0.30232558,  0.1627907 ,
         0.02325581, -0.30232558,  0.11627907, -0.11627907,  0.11627907,
        -0.02325581, -0.02325581, -0.06976744,  0.02325581, -0.06976744,
         0.1627907 , -0.34883721, -0.06976744,  0.02325581,  0.1627907 ,
        -0.20930233,  0.1627907 , -0.30232558, -0.02325581,  0.25581395,
        -0.20930233,  0.06976744,  0.06976744, -0.11627907,  0.06976744,
        -0.02325581, -0.11627907, -0.02325581, -0.1627907 ,  0.1627907 ,
        -0.11627907],
       [-0.17410956,  0.27272727, -0.09090909,  0.27272727,  0.21212121,
        -0.15151515, -0.13513514,  0.18918919,  0.24324324, -0.11627907,
         0.1627907 , -0.34883721,  0.20930233, -0.11627907,  0.06976744,
        -0.02325581, -0.20930

In [79]:
pred_test_set_inverted

array([[ 0.50582873, -2.        ,  4.        ,  3.        , -3.        ,
        -5.        ,  1.        ,  2.        , -2.        ,  4.        ,
        -7.        ,  5.        , -2.        ,  2.        ,  0.        ,
        -4.        ,  1.        ,  7.        , -6.        ,  4.        ,
         1.        , -6.        ,  3.        , -2.        ,  3.        ,
         0.        ,  0.        , -1.        ,  1.        , -1.        ,
         4.        , -7.        , -1.        ,  1.        ,  4.        ,
        -4.        ,  4.        , -6.        ,  0.        ,  6.        ,
        -4.        ,  2.        ,  2.        , -2.        ,  2.        ,
         0.        , -2.        ,  0.        , -3.        ,  4.        ,
        -2.        ],
       [-3.37280779,  4.        , -2.        ,  4.        ,  3.        ,
        -3.        , -5.        ,  1.        ,  2.        , -2.        ,
         4.        , -7.        ,  5.        , -2.        ,  2.        ,
         0.        , -4.     

In [80]:
#create dataframe that shows the predicted sales
result_list = []
sales_dates = list(df[-30:].YYMM)
act_sales = list(df[-30:].CNT)
for index in range(0,len(pred_test_set_inverted)):
    result_dict = {}
    result_dict['pred_CNT'] = int(pred_test_set_inverted[index][0] + act_sales[index])
    result_dict['YYMM'] = sales_dates[index+1]
    result_list.append(result_dict)
df_result = pd.DataFrame(result_list)
#for multistep prediction, replace act_sales with the predicted sales

In [81]:
df_result
df_sales_pred = pd.merge(df,df_result,on='YYMM',how='left')
df_result


,pred_CNT,YYMM
0,8,2021-11-23
1,2,2021-11-24
2,14,2021-11-25
3,1,2021-11-26
4,8,2021-11-29
5,6,2021-11-30


In [82]:
#merge with actual sales dataframe
df_sales_pred = pd.merge(df,df_result,on='YYMM',how='left')
#plot actual and predicted
plot_data = [
    go.Scatter(
        x=df_sales_pred['YYMM'],
        y=df_sales_pred['CNT'],
        name='actual'
    ),
        go.Scatter(
        x=df_sales_pred['YYMM'],
        y=df_sales_pred['pred_CNT'],
        name='predicted'
    )
    
]
plot_layout = go.Layout(
        title='Sales Prediction'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [84]:
list(df[-30:].YYMM)

[Timestamp('2021-11-22 00:00:00'),
 Timestamp('2021-11-23 00:00:00'),
 Timestamp('2021-11-24 00:00:00'),
 Timestamp('2021-11-25 00:00:00'),
 Timestamp('2021-11-26 00:00:00'),
 Timestamp('2021-11-29 00:00:00'),
 Timestamp('2021-11-30 00:00:00'),
 Timestamp('2021-12-01 00:00:00'),
 Timestamp('2021-12-02 00:00:00'),
 Timestamp('2021-12-03 00:00:00'),
 Timestamp('2021-12-06 00:00:00'),
 Timestamp('2021-12-07 00:00:00'),
 Timestamp('2021-12-08 00:00:00'),
 Timestamp('2021-12-09 00:00:00'),
 Timestamp('2021-12-10 00:00:00'),
 Timestamp('2021-12-13 00:00:00'),
 Timestamp('2021-12-14 00:00:00'),
 Timestamp('2021-12-15 00:00:00'),
 Timestamp('2021-12-16 00:00:00'),
 Timestamp('2021-12-17 00:00:00'),
 Timestamp('2021-12-18 00:00:00'),
 Timestamp('2021-12-20 00:00:00'),
 Timestamp('2021-12-21 00:00:00'),
 Timestamp('2021-12-22 00:00:00'),
 Timestamp('2021-12-23 00:00:00'),
 Timestamp('2021-12-24 00:00:00'),
 Timestamp('2021-12-28 00:00:00'),
 Timestamp('2021-12-29 00:00:00'),
 Timestamp('2021-12-